<a href="https://colab.research.google.com/github/STKalinowski/EncoderDecoderModelBetterGen/blob/main/t5DecoderInputInfluence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
Looking at how to imporve generation by modifying & seperating parts of the decoder encoder input.
Models like T5 are encoder-decoder models. If we want to prompt a structure, shouldn't that part only be in the decoder part?

We can understand more about encoder-decoder architecture here: https://huggingface.co/blog/encoder-decoder#encoder-decoder
'''

"\nLooking at how to imporve generation by modifying & seperating parts of the decoder encoder input.\nModels like T5 are encoder-decoder models. If we want to prompt a structure, shouldn't that part only be in the decoder part?\n\nWe can understand more about encoder-decoder architecture here: https://huggingface.co/blog/encoder-decoder#encoder-decoder\n"

In [2]:
!pip install transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 8.7 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig
import torch
from IPython.display import display, HTML

if torch.cuda.is_available():
  device='cuda'
else:
  device = 'cpu'
modelName = 'google/flan-t5-large'

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained(modelName).to(device)
tokenizer = AutoTokenizer.from_pretrained(modelName)


#####

In [26]:
text = '''
Front: The Arcane Glade

Description: Hidden deep in the heart of an untamed jungle, the Arcane Glade holds the world's most comprehensive collection of magical literature. The Glade is no ordinary library. Its endless rows of towering bookshelves are woven from twisted vines and living foliage, growing from the verdant landscape itself. Ancient texts, protected by powerful enchantments, rest within niches in the bark, nestled alongside luminescent flowers that softly illuminate the glade.
Cast:

Iramis the Bibliocurator: The elven keeper.
The Whispering Willows: Sentient tree librarians.
Zephyra, the Wind Serpent: A guardian serpent.
Dangers:

Blight of the Bound Word:
Impulse: To consume knowledge
Grim Portents: Withering books, trees losing sentience, Iramis' illness, the dying Glade.
Doom: Destruction of the Glade and loss of knowledge.
The Wind Serpent's Fury:
Impulse: To protect its bond
Grim Portents: Signs of wrath, Zephyra's barrier, Iramis' disappearance, attack on Glade.
Doom: Zephyra's tyranny over the Glade.
'''

In [27]:
# Define the input sequence and initial decoded sequence

input_sequence = f"{text}\n "
decoded_sequence = "A Bob, a human fighter, sets out to find the library, "
genConf = GenerationConfig.from_pretrained(modelName, top_p=0.98, temperature=0.7, do_sample=True, max_length=750, max_new_tokens=500, repetition_penalty=1.4)
print(genConf)

# Normal Version
input_ids = tokenizer.encode(input_sequence, return_tensors='pt', add_special_tokens=False).to(device)
normalOutput = model.generate(input_ids, generation_config=genConf)
normalRes = tokenizer.decode(normalOutput[0], skip_special_tokens=True)

# Normal with Decoder Part Included
input_ids = tokenizer.encode(input_sequence+decoded_sequence, return_tensors='pt', add_special_tokens=False).to(device)
normalDecOutput = model.generate(input_ids, generation_config=genConf)
normalDecRes = tokenizer.decode(normalDecOutput[0], skip_special_tokens=True)

# Decoder influenced
input_ids = tokenizer.encode(input_sequence, return_tensors='pt').to(device)
decoder_input_ids = tokenizer.encode(decoded_sequence, return_tensors='pt', add_special_tokens=False).to(device)
decodeOutput = model.generate(input_ids, decoder_input_ids=decoder_input_ids, generation_config=genConf)
decodeRes = tokenizer.decode(decodeOutput[0],skip_special_tokens=True)

display(HTML(f'<div><h4>Inputs:</h4><p>Task:</p><p> {input_sequence}</p><p>Decoder Input:</p><p> {decoded_sequence}</p><h4>Normal Generation:</h4><p>{normalRes}</p><br/><h4>Normal With Decoded Appended:</h4><p>{normalDecRes}</p><br/><h4>DecodedSplit:</h4> <p>{decodeRes}</p> </div>'))

GenerationConfig {
  "_from_model_config": true,
  "decoder_start_token_id": 0,
  "do_sample": true,
  "eos_token_id": 1,
  "max_length": 750,
  "max_new_tokens": 500,
  "pad_token_id": 0,
  "repetition_penalty": 1.4,
  "temperature": 0.7,
  "top_p": 0.98,
  "transformers_version": "4.30.2"
}

